In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import snowflake.connector
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [3]:
load_dotenv()

# Retrieve Snowflake credentials from environment variables
user = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = 'SF_WH_CASE1'
database = 'SF_DB_CASE1'
schema = 'SF_CASE1'
role = 'SYSADMIN'  # Replace with your full access role

# Configure SQLAlchemy engine for Snowflake
engine = create_engine(URL(
    account=account,
    user=user,
    password=password,
    database=database,
    schema=schema,
    warehouse=warehouse,
    role=role,
))

# Ensure the Snowflake setup is completed successfully.
# This part can be moved to a separate setup script if it doesn't need to run every time.
with engine.connect() as conn:
    conn.execute("CREATE DATABASE IF NOT EXISTS SF_DB_CASE1")
    conn.execute("USE DATABASE SF_DB_CASE1")
    conn.execute("""
        CREATE WAREHOUSE IF NOT EXISTS SF_WH_CASE1
        WITH WAREHOUSE_SIZE = 'MEDIUM'
        WAREHOUSE_TYPE = 'STANDARD'
        AUTO_SUSPEND = 300
        AUTO_RESUME = TRUE;
    """)
    conn.execute("CREATE SCHEMA IF NOT EXISTS SF_CASE1")
    conn.execute("USE SCHEMA SF_CASE1")
    conn.execute("""
        CREATE OR REPLACE TABLE SUMMARY_FINAL (
            NAME_OF_THE_TOPIC VARCHAR(500),
            YEAR VARCHAR(255),
            LEVEL VARCHAR(255),
            INTRODUCTION_SUMMARY VARCHAR(10000),
            LEARNING_OUTCOMES VARCHAR(10000),
            LINK_TO_THE_SUMMARY_PAGE VARCHAR(1000),
            LINK_TO_THE_PDF_FILE VARCHAR(1000)
        );
    """)
    print("Snowflake setup completed successfully.")

# Load CSV data into DataFrame
csv_file_path = '../Webscrape/CSV/extracted_updated.csv'  # Adjust this path as needed
df = pd.read_csv(csv_file_path)

# Prepare DataFrame for Snowflake
df.columns = [col.upper() for col in df.columns]

# Transfer data from CSV to Snowflake
table_name = 'SUMMARY_FINAL'
try:
    df.to_sql(name=table_name, con=engine, index=False, if_exists='append', method='multi')
    print("Data transfer to Snowflake completed successfully.")
except Exception as e:
    print(f"Failed to transfer data to Snowflake: {e}")


Snowflake setup completed successfully.


Data transfer to Snowflake completed successfully.


/Users/anirudhajoshi/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:1666: UserWarning: The provided table name 'SUMMARY_FINAL' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
